In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('precision', 2)
from QuantLib import *

## Parameter
------------

In [ ]:
maturity_date = Date(31, 1, 2018)
start_date = Date(1, 1, 2018)

spot_price = 14500
lower_strike = 14000
lower_bound = 12400
upper_strike = 14800
upper_bound = 16000

volatility = 0.30
risk_free_rate = 0.
dividend_rate = 0.
day_count = Actual365Fixed()
calendar = China(China.SSE)

## Market Data
-------------

In [ ]:
calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

In [ ]:
put_payoff1 = PlainVanillaPayoff(Option.Put, lower_strike)
put_payoff2 = PlainVanillaPayoff(Option.Put, lower_bound)
call_payoff1 = PlainVanillaPayoff(Option.Call, upper_strike)
call_payoff2 = PlainVanillaPayoff(Option.Call, upper_bound)

In [ ]:
exercise = EuropeanExercise(maturity_date)
put_option1 = VanillaOption(put_payoff1, exercise)
put_option2 = VanillaOption(put_payoff2, exercise)
call_option1 = VanillaOption(call_payoff1, exercise)
call_option2 = VanillaOption(call_payoff2, exercise)

In [ ]:
%%time
engine = AnalyticEuropeanEngine(bsm_process)
put_option1.setPricingEngine(engine)
put_option2.setPricingEngine(engine)
call_option1.setPricingEngine(engine)
call_option2.setPricingEngine(engine)

bsm_price = put_option1.NPV() - put_option2.NPV() + call_option1.NPV() - call_option2.NPV()
print("BSM european theoreticl price is {0:.4f}".format(bsm_price))

In [ ]:
## Scenario Analysis

spot_scenarios = [13500, 14000, 14500, 15000]
vol_scenarios = [0.20, 0.30, 0.40]

price_tables = np.zeros((len(spot_scenarios), len(vol_scenarios)))

for i, spot in enumerate(spot_scenarios):
    for j, vol in enumerate(vol_scenarios):
        spot_handle.linkTo(SimpleQuote(spot))
        vol_handle.linkTo(SimpleQuote(vol))
        
        bsm_price = put_option1.NPV() - put_option2.NPV() + call_option1.NPV() - call_option2.NPV()
        price_tables[i, j] = bsm_price

In [ ]:
df = pd.DataFrame(price_tables, index=spot_scenarios, columns=['vol - {0:=}%'.format(v*100) for v in vol_scenarios])
df.index.name = 'spot'

In [ ]:
print("向下保护区间：{0} ~ {1}".format(lower_bound, lower_strike))
print("向上保护区间：{0} ~ {1}".format(upper_strike, upper_bound))
df